In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

directory = './'
files = [f for f in os.listdir(directory) if f.endswith('.txt')]

plt.figure(figsize=(10, 8))

for file in files:
    data = pd.read_csv(file, sep='\t', skiprows=1,
                       names=['freq/Hz', 'Re(Z)/Ohm', 'Im(Z)/Ohm'])

    # Step 1: 翻转Im
    data['Im(Z)/Ohm'] = -data['Im(Z)/Ohm']

    # ---- Step 2: 使用原始翻转后的数据寻找符号变化区间 ----
    x = data['Re(Z)/Ohm'].values
    y = data['Im(Z)/Ohm'].values

    # 找 y 过0的区间
    sign_change_idx = np.where(np.diff(np.sign(y)) != 0)[0]

    if len(sign_change_idx) > 0:
        idx = sign_change_idx[0]
        
        # 插值区间
        x1, x2 = x[idx], x[idx + 1]
        y1, y2 = y[idx], y[idx + 1]

        # ---- Step 3: 线性插值求 Im=0 对应的 Re ----
        x_intercept = x1 - y1 * (x2 - x1) / (y2 - y1)

        # ---- Step 4: 将交点加入 DataFrame ----
        intercept_df = pd.DataFrame({
            'freq/Hz': [np.nan],         # 频率未知，可放空
            'Re(Z)/Ohm': [x_intercept],
            'Im(Z)/Ohm': [0]
        })

        data = pd.concat([data, intercept_df], ignore_index=True)

    # ---- Step 5: 删除所有 Im<0 的点（确保从0开始） ----
    data = data[data['Im(Z)/Ohm'] >= 0]

    # 按 Re 排序，避免折线跳点
    data = data.sort_values('Re(Z)/Ohm')

    # ---- Step 6: 作图 ----
    plt.plot(
        data['Re(Z)/Ohm'],
        data['Im(Z)/Ohm'],
        label=file,
        marker='o',
        linestyle='-',
        linewidth=2,
        markersize=6
    )

plt.title('Cole–Cole Plot (Interpolated Zero Crossing)')
plt.xlabel('Re(Z) / Ohm')
plt.ylabel('Im(Z) / Ohm')
plt.legend()
plt.grid(True)
plt.show()